In [1]:
import pandas as pd
import matplotlib.pyplot as plt
wd = %pwd
import os
fdel = os.path.sep

from MacroBackend import Utilities, PriceImporter, Pull_Data, Charting
import datetime

keys = Utilities.api_keys()
print(keys.keys)
myFredAPI_key = keys.keys['fred']
DataStart = "2021-01-01"
EndDateStr = datetime.date.today().strftime("%Y-%m-%d")


/Users/jamesbishop/Documents/Python/Bootleg_Macro/MacroBackend /Users/jamesbishop/Documents/Python/Bootleg_Macro


ModuleNotFoundError: No module named 'rpy2'

In [ ]:
btfp_meta, btfp = PriceImporter.PullFredSeries("H41RESPPALDKNWW",myFredAPI_key,start=DataStart,end=EndDateStr,Con2Bil=True)
btfp_d = btfp.resample('D').ffill()
print(btfp_d)
btfp_d

In [ ]:
from yahoofinancials import YahooFinancials as yf
import datetime

end_date = datetime.date.today().strftime("%Y-%m-%d")
ticker = yf("TSLA")
#data = ticker.get_historical_price_data("2000-01-01", end_date, "daily")

#data

ticker.get_summary_data()